# import

In [1]:
import time
import win32com.client
import pandas as pd

일단위 데이터와 분단위 데이터를 선택해서 가져올 수 있는 함수입니다.

분단위데이터는 2년치정도 제공하는 것으로 보입니다.

분단위 데이터를 모두 가져오는 데 25 초정도 걸리고

일단위 데이터는 1초 안으로 가져옵니다.

한 요청당 최대 2856개를 가져올 수 있고

15초당 60건을 요청할 수 있습니다.

`2856 * 60 = 171360`

# object

In [2]:
class CREON(object):
    """대신증권 크레온 API"""
    
    def __init__(self):
        # 연결 여부 체크
        self.objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = self.objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            exit()
     
    def setMethod(self, code, char, from_yyyymmdd=None, to_yyyymmdd=None, count=None):
        """
        count는 보통 상식의 데이터 개수가 아니다.
        여기서는 한번 요청 시 가져와지는 데이터의 개수이다.
        한번 요청 시 최대 2856개 가능하다.
        
        원하는 데이터 개수가 있으면 to_yyyymmdd 로 가져온 다음에 잘라서 사용한다.
        하루에 분단위 데이터가 381개이다. (* 마지막 10분은 동시호가)
        
        """
        # object 구하기
        self.objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
        self.objStockChart.SetInputValue(0, code)  # 종목코드
        
        if to_yyyymmdd:
            self.objStockChart.SetInputValue(1, ord('1'))  # 요청 구분 '1': 기간, '2': 개수
            self.objStockChart.SetInputValue(2, from_yyyymmdd)  # To 날짜
            self.objStockChart.SetInputValue(3, to_yyyymmdd)  # From 날짜
        elif count:
            self.objStockChart.SetInputValue(1, ord('2'))  # 개수로 받기
            self.objStockChart.SetInputValue(4, count)  # 조회 개수
        else: raise print("기간을 입력해주세요.")
            
        self.objStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8])  # 요청항목 - 날짜, 시간,시가,고가,저가,종가,거래량
        self.objStockChart.SetInputValue(6, ord(char))  # '차트 주기 - 분/틱
        self.objStockChart.SetInputValue(7, 1)  # 분틱차트 주기
        
        self.objStockChart.SetInputValue(9, ord('1'))  # 수정주가 사용
        
        
        
        self.data = {
            "date": [],
            "time": [],
            "open": [],
            "high": [],
            "low": [],
            "close": [],
            "vol": [],
        }
        
    def checkRequest(self):
        
        self.objStockChart.BlockRequest()
        
        rqStatus = self.objStockChart.GetDibStatus()
        
        if rqStatus != 0: 
            
            return False
        
#         else:
#             print("통신상태 양호, 누적 개수 {}".format(len(self.data["date"])))
        
        self.count = self.objStockChart.GetHeaderValue(3)
        
        if self.count <= 1: 
            
            return False
        
        return int(self.count)
    
    def checkRemainTime(self):
        
        # 연속 요청 가능 여부 체크
        remainTime = self.objCpCybos.LimitRequestRemainTime / 1000.
        remainCount = self.objCpCybos.GetLimitRemainCount(1)  # 시세 제한
        
        if remainCount <= 0:
            print("15초당 60건으로 제한합니다.")
            time.sleep(remainTime)
            
    
    def getStockPriceMin(self):
        
        while 1:
        
            self.checkRemainTime()
            rows = self.checkRequest()

            if rows:

                for i in range(rows):

                    self.data["date"].append(self.objStockChart.GetDataValue(0, i))
                    self.data["time"].append(self.objStockChart.GetDataValue(1, i))
                    self.data["open"].append(self.objStockChart.GetDataValue(2, i))
                    self.data["high"].append(self.objStockChart.GetDataValue(3, i))
                    self.data["low"].append(self.objStockChart.GetDataValue(4, i))
                    self.data["close"].append(self.objStockChart.GetDataValue(5, i))
                    self.data["vol"].append(self.objStockChart.GetDataValue(6, i))
            else:

                break
                
    
        return self.data

# run

In [3]:
creon = CREON()

In [4]:
creon.setMethod(code="A005930", char="m", from_yyyymmdd=20200101, to_yyyymmdd=10000101)

In [5]:
%%time
samsung = creon.getStockPriceMin()

15초당 60건으로 제한합니다.
Wall time: 24.1 s


In [6]:
tmp = pd.DataFrame(samsung)
tmp.tail()

,date,time,open,high,low,close,vol
189232,20170213,905,37800,37880,37800,37860,48100
189233,20170213,904,37880,37900,37780,37780,80500
189234,20170213,903,37880,37900,37840,37880,61400
189235,20170213,902,37920,37980,37820,37860,98000
189236,20170213,901,37740,37940,37720,37940,453900


In [7]:
creon.setMethod(code="A005930", char="D", from_yyyymmdd=20200101, to_yyyymmdd=10000101)

In [8]:
%%time
samsung = creon.getStockPriceMin()

Wall time: 607 ms


In [9]:
tmp = pd.DataFrame(samsung)
tmp.tail()

,date,time,open,high,low,close,vol
10541,19800109,0,38,41,37,41,1243500
10542,19800108,0,37,37,36,37,1344000
10543,19800107,0,35,35,35,35,604000
10544,19800105,0,32,34,32,34,393500
10545,19800104,0,34,34,33,33,131500
